# Въведение в линейното предсказване и речевите сигнали


### Задача за изпълнение
#### Анализиране на речеви сигнал с LPC

#### Import required libraries

In [ ]:
import numpy as np
import scipy.io.wavfile as wavfile
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt
from IPython.display import Audio

#### **Step 1. Load WAV file**
- Load WAV file: The speech signal is loaded from the specified file path, extracting the sampling rate and audio signal.
- Convert to Mono: If the audio is stereo (two channels), the code averages the two channels to create a mono signal. This simplifies the analysis and processing since LPC typically works with single-channel audio.


In [ ]:
# Replace it with a correct local path
file_path = 'speech_samples/???????????'  
sampling_rate, signal = wavfile.read(file_path)

# Convert stereo to mono if necessary.
# Check if the signal has more than one channel
if len(signal.shape) > 1:
    # Average the two channels
    signal = np.mean(signal, axis=1)

#### **Step 2. Display basic information**
- Basic Information: This section prints key details about the loaded audio signal:
    - Sampling Rate: The number of samples per second.
    - Signal Length: The total number of samples in the audio signal.
    - Signal Shape and Data Type: Provides information on the array's dimensions and the type of data it contains.
- Duration Calculation: The duration of the audio in seconds is calculated by dividing the total number of samples by the sampling rate.
- Energy Calculation: The energy of the signal is computed as the sum of the squares of the signal's amplitudes, providing an indication of the signal's strength.

In [ ]:
print(f"Sampling Rate: {sampling_rate} Hz")
print(f"Signal Length: {len(signal)} samples")
print(f"Signal shape: {signal.shape}, Data type: {signal.dtype}")

# Calculate the duration of the signal in seconds
duration = len(signal) / sampling_rate
print(f"Duration: {duration:.2f} seconds")

# Calculate the energy of the signal
energy = sum(signal**2)
print(f"Energy of the Signal: {energy}")

#### **Step 3. Compute the LPC residual**
- Define LPC Order: The variable p sets the order of the LPC model, which usually ranges from 10 to 20 for speech signals.
- Auto-correlation Function: A custom function, autocorr, calculates the auto-correlation of the signal for a specified number of lags. This is essential for determining how similar the signal is to itself at different time intervals.
- Compute Auto-correlation Sequence: The auto-correlation sequence is computed for the first p lags.
- Calculate LPC Coefficients: The LPC coefficients are calculated using the Levinson-Durbin algorithm, which efficiently solves the normal equations derived from the auto-correlation sequence. The first coefficient is set to 1 by convention.
- Predicted Signal Calculation: The predicted signal is computed using the LPC coefficients by iterating through the original signal and applying the LPC model.
- Residual Calculation: The residual is the difference between the original signal and the predicted signal, representing the prediction error.

In [ ]:
p = 12  # typically 10-20 for speech

# Calculate the auto-correlation of the signal
def autocorr(x, lag):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:len(result)//2+lag]

# Compute the auto-correlation sequence for the first p lags
r = autocorr(signal, p+1)

# Solve the normal equations using Levinson-Durbin recursion to get LPC coefficients
# r[0] is the energy, r[1:p+1] are the autocorrelations
a = np.linalg.solve(toeplitz(r[:p]), -r[1:p+1])
a = np.concatenate(([1], a))  # the first coefficient is always 1

print(f"LPC Coefficients: \n{a}")

# Compute the LPC predicted signal
predicted_signal = np.zeros(len(signal))
for n in range(p, len(signal)):
    predicted_signal[n] = sum(a[i] * signal[n - i - 1] for i in range(p))

# Compute the LPC residual (prediction error)
residual = signal - predicted_signal

#### **Step 4. Create a double plot (subplots)**
- Normalization: The residual and difference signals are normalized to a range of [-1, 1] for better visualization in plots.
- Subplot Creation: A figure with three subplots is created to visualize:
    - The original speech signal.
    - The LPC residual signal, showing the prediction error.
    - The difference signal between the original and predicted signals.
- Layout Adjustment: The tight_layout function is called to ensure that subplots do not overlap.

In [ ]:
# Normalize residual for plotting
residual = residual / np.max(np.abs(residual))  # Normalize to the range [-1, 1]

# Compute the difference between the original signal and predicted signal
difference = signal - predicted_signal  # This is the correct difference signal

# Normalize the difference for plotting
difference = difference / np.max(np.abs(difference))  # Normalize to the range [-1, 1]

plt.figure(figsize=(12, 12))  # Increased figure height for better clarity

# Plot original signal
plt.subplot(3, 1, 1)  # 3 rows, 1 column, 1st subplot
plt.plot(signal, color='blue')
plt.title('Original Speech Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.grid()

# Plot LPC residual signal
plt.subplot(3, 1, 2)  # 3 rows, 1 column, 2nd subplot
plt.plot(residual, color='orange')
plt.title('LPC Residual Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.grid()

# Plot difference between original signal and predicted signal
plt.subplot(3, 1, 3)  # 3 rows, 1 column, 3rd subplot
plt.plot(difference, color='green')
plt.title('Difference between Original Signal and Predicted Signal')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.grid()

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

#### **Step 5. Save the LPC residual to a new WAV file**
- File Naming: The code constructs a new filename for the LPC residual by appending _lpc_residual to the original file name (without extension).
- Saving the Residual: The residual signal is saved as a new WAV file using 16-bit PCM format, allowing for easy playback and further analysis.

In [ ]:
# Generate new filename based on the original file name
original_file_name = file_path.split('/')[-1].split('.')[0]  # Extract the original filename without extension
output_file_name = f"{original_file_name}_lpc_residual.wav"  # New filename
output_file_path = f"speech_samples/{output_file_name}"  # Specify the output file path

# Save as 16-bit PCM
wavfile.write(output_file_path, sampling_rate, (residual * 32767).astype(np.int16))  
print(f"LPC residual saved as: {output_file_path}")

#### **Step 6.1. Save the LPC residual to a new WAV file**
- Original Signal Playback: The original audio signal is prepared for playback, allowing the user to listen to it directly in the Jupyter notebook.

In [ ]:
# Task 6.1: Playback options for both original and LPC residual signals
print("Playback of the original signal:")

# Original audio playback
Audio(signal, rate=sampling_rate, normalize=True)

#### **Стъпка 6.2. Save the LPC residual to a new WAV file**
- LPC Residual Playback: Similarly, the LPC residual signal is prepared for playback, allowing the user to listen to the audio after LPC processing.

In [ ]:
# Task 6.1: Playback options for both original and LPC residual signals
print("Playback of the LPC residual audio signal:")

# LPC residual audio playback
Audio(residual, rate=sampling_rate, normalize=True)